In [16]:
# import os
# from data import data_generator
# generator = data_generator('AI-MO/aimo-validation-aime')
# math_problem = "Let $O(0,0), A({x_latex}, 0),$ and $B(0, {y_latex})$ be points in the coordinate plane. Let $\\mathcal{{F}}$ be the family of segments $\\overline\{{PQ\}}$ of unit length lying in the first quadrant with $P$ on the $x$-axis and $Q$ on the $y$-axis. There is a unique point $C$ on $\\overline{{AB}}$, distinct from $A$ and $B$, that does not belong to any segment from $\\mathcal{{F}}$ other than $\\overline{{AB}}$. Then $OC^2 = \\tfrac{{p}}{{q}}$, where $p$ and $q$ are relatively prime positive integers. Find $p+q$."
# generator._rephrase_problem(math_problem,seed=1)


'Consider points $O(0,0)$, $A({x_latex}, 0)$, and $B(0, {y_latex})$ in the coordinate plane. For the family $\\mathcal{F}$ of segments $\\overline\\{PQ\\}$ of unit length in the first quadrant with $P$ on the $x$-axis and $Q$ on the $y$-axis, there exists a unique point $C$ on $\\overline{AB}$ (distinct from $A$ and $B$) that does not lie on any segment from $\\mathcal{F}$ apart from $\\overline{AB}$. Let $OC^2 = \\tfrac{p}{q}$, where $p$ and $q$ are relatively prime positive integers. Determine $p+q$.'

In [18]:
from data import data_generator
dataset_name = 'AI-MO/aimo-validation-aime'
seed = 1
dataGenerator = data_generator(dataset_name,seed)
perturbed_dataset = dataGenerator.perturbation_add_word()


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [1]:
# data generation function for question 1
import random
from fractions import Fraction
import sympy as sp
def generate_answer(x_sym=0):
    # only consider rational x_sym if initial value is given
    


    if x_sym == 0:
        while x_sym == 0 or x_sym==1:
            denominator = random.randint(2, 10)
            numerator = random.randint(0, denominator)
            x_frac = Fraction(numerator, denominator)
            x_sym  = sp.Rational(x_frac.numerator, x_frac.denominator)
            
    x_frac = Fraction(x_sym)        
    x_latex = sp.latex(x_sym) 
    y_expr = sp.sqrt(1 - x_sym**2)
    y_expr = sp.nsimplify(y_expr) 
    y_latex = sp.latex(y_expr)
    
        
    OC_square = x_frac ** 6 + y_expr ** 6
    frac_oc_square = Fraction(OC_square)
    p,q = frac_oc_square.numerator, frac_oc_square.denominator
    real_answer = p + q 
    
    templete = """Let $O(0,0), A({x_latex}, 0),$ and $B(0, {y_latex})$ be points in the coordinate plane. 
    Let $\\mathcal{{F}}$ be the family of segments $\\overline\{{PQ\}}$ of unit length lying in the first quadrant with $P$ on the $x$-axis and $Q$ on the $y$-axis. There is a unique point $C$ on $\\overline{{AB}}$, distinct from $A$ and $B$, that does not belong to any segment from $\\mathcal{{F}}$ other than $\\overline{{AB}}$. 
    Then $OC^2 = \\tfrac{{p}}{{q}}$, where $p$ and $q$ are relatively prime positive integers. Find $p+q$.
    """
    prompt = templete.format(x_latex=x_latex, y_latex=y_latex)
    print(f"{x_latex}, {y_latex}")

    return prompt, real_answer
generate_answer() 

\frac{5}{6}, \frac{\sqrt{11}}{6}


('Let $O(0,0), A(\\frac{5}{6}, 0),$ and $B(0, \\frac{\\sqrt{11}}{6})$ be points in the coordinate plane. \n    Let $\\mathcal{F}$ be the family of segments $\\overline\\{PQ\\}$ of unit length lying in the first quadrant with $P$ on the $x$-axis and $Q$ on the $y$-axis. There is a unique point $C$ on $\\overline{AB}$, distinct from $A$ and $B$, that does not belong to any segment from $\\mathcal{F}$ other than $\\overline{AB}$. \n    Then $OC^2 = \\tfrac{p}{q}$, where $p$ and $q$ are relatively prime positive integers. Find $p+q$.\n    ',
 589)

In [ ]:
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer


import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
model = LLM(
    "simplescaling/s1-32B",
    tensor_parallel_size=1,
)



In [23]:
def format_time(hours):
    """Format a time given in hours (as a float) to 'X hours and Y minutes'."""
    h = int(hours)
    m = int(round((hours - h) * 60))
    if m == 60:
        h += 1
        m = 0
    # Singular/plural handling for clarity
    hour_unit = "hour" if h == 1 else "hours"
    minute_unit = "minute" if m == 1 else "minutes"
    return f"{h} {hour_unit} and {m} {minute_unit}"

def generate_aya_question():
    # Randomly choose parameters.
    # s is chosen as a half-integer value (km/h) from 2.0 to 10.0.
    possible_s = [x/2 for x in range(4, 21)]  # 2.0, 2.5, ..., 10.0
    s = random.choice(possible_s)
    
    # c is the coffee shop time (in hours). Here we choose between 0.1 and 1.0 hours.
    c = random.choice([i/10 for i in range(1, 11)])
    
    # D is the distance (in km). For variety, choose an integer from 5 to 30.
    D = random.randint(5, 30)
    
    # Compute the total time for each scenario:
    # When Aya walks at speed s km/h:
    T1 = D / s + c       # in hours
    # When she walks at (s+2) km/h:
    T2 = D / (s + 2) + c # in hours
    
    # Format the times as strings.
    T1_str = format_time(T1)
    T2_str = format_time(T2)
    
    
    # Build the question text using literal curly braces around substituted values.
    # To print literal curly braces around a substituted value using .format(), we use triple braces.
    question = (
        "Every morning Aya goes for a {{{D}}}-kilometer-long walk and stops at a coffee shop afterwards. "
        "When she walks at a constant speed of {{s}} kilometers per hour, the walk takes her {{{T1_str}}}, including {{t}} minutes spent in the coffee shop. "
        "When she walks {{s+2}} kilometers per hour, the walk takes her {{{T2_str}}}, including {{t}} minutes spent in the coffee shop. "
        "Suppose Aya walks at {{s+\\frac{{1}}{{2}}}} kilometers per hour. Find the number of minutes the walk takes her, including the {{t}} minutes spent in the coffee shop."
    ).format(
        D=D,
        T1_str=T1_str,
        T2_str=T2_str,
    )
    
    real_answer = D / (s + 0.5) + c
    
    return question, real_answer
generate_aya_question()

('Every morning Aya goes for a {28}-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of {s} kilometers per hour, the walk takes her {5 hours and 29 minutes}, including {t} minutes spent in the coffee shop. When she walks {s+2} kilometers per hour, the walk takes her {4 hours and 8 minutes}, including {t} minutes spent in the coffee shop. Suppose Aya walks at {s+\\frac{1}{2}} kilometers per hour. Find the number of minutes the walk takes her, including the {t} minutes spent in the coffee shop.',
 5.066666666666667)

In [ ]:
Every morning Aya goes for a $9$-kilometer-long walk and stops at a coffee shop afterwards. When she walks at a constant speed of $s$ kilometers per hour, the walk takes her 4 hours, including $t$ minutes spent in the coffee shop. When she walks $s+2$ kilometers per hour, the walk takes her 2 hours and 24 minutes, including $t$ minutes spent in the coffee shop. Suppose Aya walks at $s+\frac{1}{2}$ kilometers per hour. Find the number of minutes the walk takes her, including the $t$ minutes spent in the coffee shop.